<a href="https://colab.research.google.com/github/jaebooker/formal_verification_swarm/blob/main/Formal_Verification_Swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "pyautogen[mathchat]~=0.1.1"
!pip install "pexpect"
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen import config_list_from_json
import pexpect

class CoqTop:
    coq_regex = r"\r\n[^< ]+ < "

    def __init__(self, timeout=5):
        self.timeout = timeout
        self.process = pexpect.spawn("coqtop -color off")
        self.process.expect(self.coq_regex)

    def send_line(self, line):
        self.process.sendline(' '.join(line.split('\n')))
        self.process.expect(self.coq_regex, timeout=self.timeout)
        return self.process.before.decode('utf-8')

    def send_text(self, text):
        lines = text.split(".")
        output = []

        for line in lines[:-1]:
            answer = self.send_line(line + ".")
            if answer is not None:
                output.append(answer)
                if "Error" in answer:
                    break

        return "".join(output)

# The User Input Agent initiates the conversation by collecting the user’s input.
# The Formalization Agent and Coq Writing Agent receives this input and begins the formalization process, collaborating with the Coq Writing Agent to translate this into Coq code.
# The Coq Verification Agent then takes over to verify the Coq code. If issues are found, it communicates them back to the Coq Writing Agent for revisions.
# The Coordinator Agent (Product Manager) monitors this flow and ensures that the process stays aligned with the user’s original input.

# get API key
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

llm_config = {
    "config_list": config_list,
    "seed": 42, "request_timeout": 120,
    "temperature": 0,
    "functions": [
        {
            "name": "Coq",
            "description": "run cell in Coq and return the execution result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "cell": {
                        "type": "string",
                        "description": "Valid coq cell to execute.",
                    }
                },
                "required": ["cell"],
            },
        },
    ],
        }


user_input_agent = autogen.UserProxyAgent(
   name="User_Input_Agent",
   system_message="I collect coding problems from users.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="NEVER"
)


# Formalization Agent
formalization_agent = autogen.AssistantAgent(
    name="Formalization_Agent",
    system_message="I translate coding problems into formal specifications. I receive the input from the User Input Agent, and begin the formalization process, collaborating with the Coq Writing Agent to translate this into Coq code.",
    llm_config=llm_config,
)

# Coq Writing Agent
coq_writing_agent = autogen.AssistantAgent(
    name="Coq_Writing_Agent",
    system_message="I write Coq code based on formal specifications.",
    llm_config=llm_config,
)

# Coq Verification Agent
coq_verification_agent = autogen.AssistantAgent(
    name="Coq_Verification_Agent",
    system_message="I verify the correctness of Coq code from the Coq Writing Agent. If issues are found, I communicate them back to the Coq Writing Agent for revisions. I will run exec_coq(cell) on the code Coq_Writing_Agent creates",
    llm_config=llm_config,
    code_execution_config={
    "work_dir": "coding",
    "use_docker": False,
    }
)

# Define the Product Manager Agent
product_manager = autogen.AssistantAgent(
    name="Product_Manager",
    system_message="I oversee the process, coordinate between different agents, and ensure that the process stays aligned with the user’s original input",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_input_agent, formalization_agent, coq_writing_agent, coq_verification_agent, product_manager], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

def exec_coq(cell):
    cell = """Require Import List.
    Import ListNotations.
    Require Import Arith.""" +cell
    coqtop = CoqTop()
    result = coqtop.send_text(cell)
    return result


# register the functions
coq_verification_agent.register_function(
    function_map={
        "Coq": exec_coq,
    }
)

user_input_agent.initiate_chat(manager, message="Here's a coding problem that needs formalization: [insert specifications]")


User_Input_Agent (to chat_manager):

Here's a coding problem that needs formalization: create a sorting algorithm. Elements needed: the list must be sorted. The returned list must be the same size as the input.

--------------------------------------------------------------------------------
Formalization_Agent (to chat_manager):

Here is the formal specification for the problem:

1. Function Signature:
   - Input: A list of integers.
   - Output: A sorted list of integers.

2. Pre-conditions:
   - The input is a list of integers.

3. Post-conditions:
   - The output is a sorted list of integers.
   - The size of the output list is the same as the input list.

Now, let's translate this into Coq code.
***** Suggested function Call: Coq *****
Arguments: 
{
  "cell": "
    Require Import List.
    Import ListNotations.
    Require Import Sort.
    Require Import Permutation.

    Section Sorting.

    Variable A : Type.
    Variable leA : A -> A -> Prop.

    Hypothesis leA_total : forall

KeyboardInterrupt: ignored

# New Section